In [ ]:
!pip install -q yfinance arch PyWavelets tqdm


import os, math, warnings, io
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import yfinance as yf
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests

import pywt  # comes from PyWavelets
from arch.univariate import arch_model


# Colab upload helper (safe to import outside Colab; will noop if missing)
try:
    from google.colab import files
    IN_COLAB = True
except Exception:
    IN_COLAB = False

# --- Output folders (daily & weekly) ---
ROOT = "expanded_outputs"
FIG_D = f"{ROOT}/figures_results_daily"
TAB_D = f"{ROOT}/tables_results_daily"
FIG_W = f"{ROOT}/figures_results_weekly"
TAB_W = f"{ROOT}/tables_results_weekly"
for p in [ROOT, FIG_D, TAB_D, FIG_W, TAB_W]:
    os.makedirs(p, exist_ok=True)

plt.rcParams['figure.dpi'] = 140
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 1200)

def savefig(path):
    plt.tight_layout()
    plt.savefig(path, bbox_inches="tight", dpi=140)
    plt.close()

def var_fit(df, maxlags=10):
    model = VAR(df)
    sel = model.select_order(maxlags=maxlags)
    p = sel.selected_orders.get('aic') or sel.selected_orders.get('bic') or sel.selected_orders.get('hqic')
    p = int(p) if p and not math.isnan(p) else 2
    return model.fit(p)

def generalized_fevd(var_res, H=10):
    A = np.stack(var_res.coefs, axis=0)  # (p,K,K)
    p, K, _ = A.shape
    Sigma = var_res.sigma_u
    F = np.zeros((K*p, K*p))
    F[:K, :K*p] = A.reshape(K, K*p)
    F[K:, :K*(p-1)] = np.eye(K*(p-1))
    J = np.zeros((K, K*p)); J[:, :K] = np.eye(K)

    Phi = [np.eye(K)]
    Fk = np.eye(K*p)
    for h in range(1, H):
        Fk = Fk @ F
        Phi.append(J @ Fk @ J.T)

    gfevd = np.zeros((K, K))
    Sigma_diag = np.diag(Sigma)
    for j in range(K):
        e_j = np.zeros((K,1)); e_j[j,0] = 1.0
        denom = Sigma_diag[j]
        for i in range(K):
            num_ij = 0.
            for h in range(H):
                phi = Phi[h]
                num_ij += (phi[i,:] @ Sigma @ e_j)**2
            gfevd[i,j] = num_ij / denom

    gfevd = gfevd / gfevd.sum(axis=1, keepdims=True)
    return gfevd

def connectedness(df, H=10, maxlags=10):
    res = var_fit(df, maxlags=maxlags)
    fevd = generalized_fevd(res, H=H)
    off = fevd.copy(); np.fill_diagonal(off, 0.0)
    TO = off.sum(axis=1)
    FROM = off.sum(axis=0)
    NET = TO - FROM
    TCI = off.sum().sum() / fevd.shape[0] * 100
    names = list(df.columns)
    fevd_tbl = pd.DataFrame(fevd, index=names, columns=names)
    dir_tbl  = pd.DataFrame({"TO_others":TO, "FROM_others":FROM, "NET":NET}, index=names)
    return fevd_tbl, dir_tbl, TCI

def swt_details(series, wavelet="db2", level=3):
    coeffs = pywt.swt(series.values, wavelet, level=level)
    details = [pd.Series(cD, index=series.index) for (cA, cD) in coeffs]
    return details  # [D1, D2, D3] with D1 = highest frequency

cols = ["BTC","ETH","SPX","DJI","GOLD"]


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 362, in run
    resolver = self.make_resolver(
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 177, in make_resolver
    return pip._internal.resolution.resolvelib.resolver.Resolver(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 58, in __init__
    self.factory = Factory(
                   ^^^^^^^^
  File "/usr/local/lib/py

KeyboardInterrupt: 

In [ ]:
RET_PATH = f"{ROOT}/merged_returns.csv"

if IN_COLAB:
    print("If you have your cleaned merged_returns.csv ready, upload it now.")
    uploaded = files.upload()  # choose your file
    if uploaded:
        # Take the first uploaded file (assume it's merged_returns.csv)
        fname = list(uploaded.keys())[0]
        df_up = pd.read_csv(io.BytesIO(uploaded[fname]), parse_dates=["Date"])
        expected = {"BTC_Close_Return","ETH_Close_Return","SPX_Close_Return","DJI_Close_Return","Gold_Close_Return"}
        if expected.issubset(set(df_up.columns)):
            df_up.to_csv(RET_PATH, index=False)
            print(f"Saved uploaded returns to {RET_PATH}")
        else:
            print("Uploaded file is missing expected columns; will fall back to Yahoo Finance.")
else:
    print("Not running in Colab; upload step skipped.")


If you have your cleaned merged_returns.csv ready, upload it now.


Saving merged_returns.csv to merged_returns (2).csv
Saved uploaded returns to expanded_outputs/merged_returns.csv


## Load daily returns
- Prefer uploaded `merged_returns.csv` to stay consistent with my Methodology.
- If not present, fallback builds from Yahoo Finance (Adj Close, 2016+).


In [ ]:
if os.path.exists(RET_PATH):
    ret = pd.read_csv(RET_PATH, parse_dates=["Date"])
    expected = {"BTC_Close_Return","ETH_Close_Return","SPX_Close_Return","DJI_Close_Return","Gold_Close_Return"}
    assert expected.issubset(set(ret.columns)), f"merged_returns.csv missing: {expected - set(ret.columns)}"
    print("Loaded daily returns from", RET_PATH)
else:
    print("No merged_returns.csv found. Building from Yahoo Finance (daily, 2016+)...")
    TICKERS = ["BTC-USD","ETH-USD","^GSPC","^DJI","GC=F"]
    px = yf.download(TICKERS, start="2016-01-01", progress=False)["Adj Close"].dropna()
    px = px.dropna()
    df = (px.reset_index()
            .rename(columns={"Date":"Date","BTC-USD":"BTC_Close","ETH-USD":"ETH_Close",
                             "^GSPC":"SPX_Close","^DJI":"DJI_Close","GC=F":"Gold_Close"}))
    df = df.sort_values("Date").reset_index(drop=True)
    for c in ["BTC_Close","ETH_Close","SPX_Close","DJI_Close","Gold_Close"]:
        df[f"{c}_Return"] = np.log(df[c] / df[c].shift(1))
    ret = df[["Date","BTC_Close_Return","ETH_Close_Return","SPX_Close_Return","DJI_Close_Return","Gold_Close_Return"]].dropna().copy()
    ret.to_csv(RET_PATH, index=False)
    print("Saved Yahoo-built daily returns to", RET_PATH)

# Simplify names (daily)
rets = ret.rename(columns={
    "BTC_Close_Return":"BTC",
    "ETH_Close_Return":"ETH",
    "SPX_Close_Return":"SPX",
    "DJI_Close_Return":"DJI",
    "Gold_Close_Return":"GOLD"
}).dropna().copy()

rets = rets.set_index(pd.to_datetime(ret["Date"]))[cols]
display(rets.tail())


Loaded daily returns from expanded_outputs/merged_returns.csv


,BTC,ETH,SPX,DJI,GOLD
Date,,,,,
2023-12-22,0.002907,0.038097,0.001659,-0.000492,0.005510
2023-12-26,-0.034168,-0.041985,0.004223,0.004254,0.001277
2023-12-27,0.022074,0.064841,0.001429,0.002957,0.002063
2023-12-28,-0.019788,-0.014288,0.000370,0.001422,0.005926
2023-12-29,-0.012241,-0.020128,-0.002830,-0.000545,-0.003648


## Weekly resample (W–FRI)
Sum of daily log returns within each week (equivalent to log(Fri / prior Fri)).


In [ ]:
rets_w = rets.resample("W-FRI").sum().dropna()
rets_w.to_csv(f"{TAB_W}/weekly_returns.csv")
display(rets_w.tail())


,BTC,ETH,SPX,DJI,GOLD
Date,,,,,
2023-12-01,0.025645,0.002740,0.007710,0.023882,-0.001308
2023-12-08,0.132680,0.122417,0.002118,0.000065,-0.009216
2023-12-15,-0.051942,-0.061187,0.024631,0.028751,0.018495
2023-12-22,0.047803,0.047459,0.007482,0.002164,0.001144
2023-12-29,-0.044124,-0.011560,0.003192,0.008087,0.005618


## DAILY — sanity visuals
Correlation matrix and rolling 60-day BTC–SPX correlation (consistency with EDA).


In [ ]:
# Correlation heatmap (daily)
corr = rets.corr()
plt.figure(figsize=(5.6,4.6))
sns.heatmap(corr, annot=True, fmt=".2f", vmin=-1, vmax=1, cmap="vlag")
plt.title("Return Correlations (Daily)")
savefig(f"{FIG_D}/corr_heatmap_daily.png")
corr.to_csv(f"{TAB_D}/correlations_daily.csv")

# Rolling 60-day correlation BTC–SPX (daily)
plt.figure(figsize=(8,4))
(rets["BTC"].rolling(60).corr(rets["SPX"])).plot()
plt.title("Rolling 60-Day Correlation: BTC vs SPX (Daily)")
plt.ylim(-1,1); plt.axhline(0, ls="--", lw=1)
savefig(f"{FIG_D}/rollcorr_btc_spx_daily.png")


## DAILY — Granger causality
Min p-value across lags 1..5 (SSR chi-square).


In [ ]:
pairs = [("BTC","SPX"), ("ETH","SPX"), ("BTC","DJI"), ("BTC","GOLD"), ("ETH","GOLD")]
rows = []
for x,y in pairs:
    data = rets[[y,x]].dropna().values  # [target, cause]
    res = grangercausalitytests(data, maxlag=5, verbose=False)
    pvals = [res[i+1][0]['ssr_chi2test'][1] for i in range(5)]
    rows.append({"cause": x, "effect": y, "min_p": float(np.min(pvals))})
g_daily = pd.DataFrame(rows).sort_values("min_p")
display(g_daily)
g_daily.to_csv(f"{TAB_D}/granger_minp_daily.csv", index=False)


,cause,effect,min_p
4,ETH,GOLD,5.019154e-07
3,BTC,GOLD,2.821087e-06
2,BTC,DJI,3.019645e-05
0,BTC,SPX,4.524227e-04
1,ETH,SPX,2.375922e-03


## DAILY — VAR → generalized FEVD connectedness (full sample)
Exports FEVD matrix, directional TO/FROM/NET, and TCI (%).


In [ ]:
fevdD, dirD, TCI_D = connectedness(rets[cols], H=10, maxlags=10)
fevdD.to_csv(f"{TAB_D}/fevd_full_daily.csv", index=True)
dirD_reset = dirD.reset_index().rename(columns={"index": "Asset"})
dirD_reset.to_csv(f"{TAB_D}/directional_full_daily.csv", index=False)
pd.DataFrame({"TCI":[TCI_D]}).to_csv(f"{TAB_D}/TCI_full_daily.csv", index=False)

print("DAILY TCI:", round(TCI_D,2), "%")
display(dirD.sort_values("NET", ascending=False))

plt.figure(figsize=(6.2,5.0))
sns.heatmap(fevdD, annot=True, fmt=".2f", cmap="rocket", cbar_kws={"label":"GFEVD share"})
plt.title("Generalized FEVD (H=10) — Full Sample (Daily)")
savefig(f"{FIG_D}/fevd_full_daily.png")


DAILY TCI: 51.47 %


,TO_others,FROM_others,NET
GOLD,0.436316,0.054003,0.382312
SPX,0.571671,0.622382,-0.050712
DJI,0.568988,0.619813,-0.050825
BTC,0.498695,0.628928,-0.130233
ETH,0.497949,0.648492,-0.150542


## DAILY — Rolling connectedness
Window ≈ 500 obs (~2y), step = 10. Saves rolling TCI and NET per asset.


In [ ]:
WINDOW, STEP, H, MAXLAGS = 500, 10, 10, 10
dates, tci_vals = [], []
net = {c: [] for c in cols}

for i in tqdm(range(WINDOW, len(rets), STEP)):
    sub = rets.iloc[i-WINDOW:i]
    try:
        _, dir_t, tci_t = connectedness(sub, H=H, maxlags=MAXLAGS)
        dates.append(rets.index[i])
        tci_vals.append(tci_t)
        for c in cols:
            net[c].append(dir_t.loc[c, "NET"])
    except Exception:
        continue

rollD = pd.DataFrame({"date":dates, "TCI":tci_vals}).set_index("date")
for c in cols:
    rollD[f"NET_{c}"] = net[c]
rollD.to_csv(f"{TAB_D}/rolling_connectedness_daily.csv")

plt.figure(figsize=(10,4))
rollD["TCI"].plot()
plt.title(f"Rolling TCI (Daily, window={WINDOW}, step={STEP})")
plt.ylabel("%")
savefig(f"{FIG_D}/rolling_TCI_daily.png")

plt.figure(figsize=(10,6))
rollD[[f"NET_{c}" for c in cols]].plot(ax=plt.gca(), lw=1)
plt.axhline(0, ls="--", lw=1)
plt.title("Rolling NET Spillovers (Daily)")
savefig(f"{FIG_D}/rolling_NET_daily.png")


100%|██████████| 51/51 [00:23<00:00,  2.20it/s]


## DAILY — Time-scale connectedness (SWT, db2, 3 levels)


In [ ]:
import pywt

def prep_for_swt_df(df_in: pd.DataFrame, desired_levels: int):
    """
    Trim the dataframe so its length is divisible by 2**L,
    with L <= desired_levels and L <= swt_max_level(len).
    Returns (df_trimmed, levels_used).
    """
    n = len(df_in)
    if n < 4:
        raise ValueError("Series too short for SWT.")

    max_allowed = pywt.swt_max_level(n)
    L = min(desired_levels, max_allowed) if max_allowed >= 1 else 1

    # Trim to multiple of 2**L; if too short, back off L until feasible
    def _trim(n, L):
        m = 2**L
        n2 = n - (n % m)
        return n2, m

    n2, m = _trim(n, L)
    while n2 < m * 10 and L > 1:  # keep at least ~10 blocks for stability
        L -= 1
        n2, m = _trim(n, L)

    if n2 == 0:
        # fall back to L=1 with even length
        L = 1
        n2, m = _trim(n, L)
        if n2 == 0:
            raise ValueError("Unable to trim to a valid length for SWT.")

    df_trim = df_in.iloc[-n2:].copy()
    return df_trim, L

def swt_details_series(series: pd.Series, levels: int, wavelet: str = "db2"):
    """
    Run SWT at a fixed number of levels on a series.
    Assumes the caller has already trimmed to a valid length.
    Returns list of detail coefficients [D1..Dlevels].
    """
    coeffs = pywt.swt(series.values, wavelet, level=levels)
    details = [pd.Series(cD, index=series.index) for (cA, cD) in coeffs]
    return details


In [ ]:
# DAILY — Time-scale connectedness (SWT) — fixed/robust
DESIRED_LEVELS = 3  # target number of levels
daily_base = rets[cols].dropna()

# Trim once so length is divisible by 2**L and all columns stay aligned
daily_trim, LEVELS_EFF = prep_for_swt_df(daily_base, DESIRED_LEVELS)
print(f"Daily SWT levels used: {LEVELS_EFF}; "
      f"trimmed to length {len(daily_trim)} (multiple of {2**LEVELS_EFF}).")

tci_lvls = []
for L in range(1, LEVELS_EFF + 1):
    # Build the level-L detail dataframe across assets
    comp = {}
    for c in cols:
        D = swt_details_series(daily_trim[c], levels=LEVELS_EFF, wavelet="db2")
        comp[c] = D[L-1]  # pick detail level L for asset c
    dfL = pd.DataFrame(comp, index=daily_trim.index).dropna()

    # Connectedness on this scale
    fevdL, dirL, tciL = connectedness(dfL, H=10, maxlags=10)
    tci_lvls.append((f"L{L}", float(tciL)))  # <-- important: store (level, TCI)

    # Save artifacts (WITH clean 'Asset' column)
    fevdL.to_csv(f"{TAB_D}/fevd_scale_L{L}_daily.csv", index=True)
    dirL_reset = dirL.reset_index().rename(columns={"index": "Asset"})
    dirL_reset.to_csv(f"{TAB_D}/directional_scale_L{L}_daily.csv", index=False)
    pd.DataFrame({"TCI": [tciL]}).to_csv(f"{TAB_D}/TCI_scale_L{L}_daily.csv", index=False)

# Plot bar chart if we collected any levels
if tci_lvls:
    levels, tcis = zip(*tci_lvls)
    plt.figure(figsize=(5.5,3.8))
    plt.bar(levels, tcis)
    plt.title("Connectedness by Time Scale (Daily)")
    plt.ylabel("%")
    savefig(f"{FIG_D}/TCI_by_scale_daily.png")
else:
    print("No admissible SWT levels; skipping daily scale plot.")


Daily SWT levels used: 1; trimmed to length 1004 (multiple of 2).


## DAILY — Optional robustness
GARCH(1,1) vols (BTC, SPX) + Rolling 1-year β (SPX on BTC).


In [ ]:
def garch_vol(ser):
    am = arch_model(ser*100, mean='zero', vol='GARCH', p=1, q=1, dist='normal')
    res = am.fit(disp='off')
    return res.conditional_volatility

btc_vol = garch_vol(rets["BTC"])
spx_vol = garch_vol(rets["SPX"])
plt.figure(figsize=(10,4))
btc_vol.plot(label="BTC vol"); spx_vol.plot(label="SPX vol")
plt.legend(); plt.title("GARCH(1,1) Conditional Volatility — Daily")
savefig(f"{FIG_D}/garch_vols_daily.png")

# Rolling 1y β: SPX ~ BTC
window=252
betas=[]
idx=rets.index
for i in range(window, len(rets)):
    y = rets["SPX"].iloc[i-window:i]
    X = sm.add_constant(rets["BTC"].iloc[i-window:i])
    b = sm.OLS(y, X).fit().params["BTC"]
    betas.append(b)
beta_ser = pd.Series(betas, index=idx[window:])
beta_ser.to_csv(f"{TAB_D}/rolling_beta_spx_on_btc_daily.csv")
plt.figure(figsize=(10,4))
beta_ser.plot(); plt.axhline(0, ls="--", lw=1)
plt.title("Rolling 1-Year β (SPX on BTC) — Daily")
savefig(f"{FIG_D}/rolling_beta_spx_on_btc_daily.png")


## WEEKLY — sanity visuals
Same as daily, but on weekly log returns (W–FRI).


In [ ]:
corrW = rets_w.corr()
plt.figure(figsize=(5.6,4.6))
sns.heatmap(corrW, annot=True, fmt=".2f", vmin=-1, vmax=1, cmap="vlag")
plt.title("Return Correlations (Weekly)")
savefig(f"{FIG_W}/corr_heatmap_weekly.png")
corrW.to_csv(f"{TAB_W}/correlations_weekly.csv")

# Rolling 26-week correlation BTC–SPX (~6 months)
plt.figure(figsize=(8,4))
(rets_w["BTC"].rolling(26).corr(rets_w["SPX"])).plot()
plt.title("Rolling 26-Week Correlation: BTC vs SPX (Weekly)")
plt.ylim(-1,1); plt.axhline(0, ls="--", lw=1)
savefig(f"{FIG_W}/rollcorr_btc_spx_weekly.png")


## WEEKLY — Granger causality
Fewer observations → use maxlag = 4.


In [ ]:
pairs = [("BTC","SPX"), ("ETH","SPX"), ("BTC","DJI"), ("BTC","GOLD"), ("ETH","GOLD")]
rows = []
for x,y in pairs:
    data = rets_w[[y,x]].dropna().values
    res = grangercausalitytests(data, maxlag=4, verbose=False)
    pvals = [res[i+1][0]['ssr_chi2test'][1] for i in range(4)]
    rows.append({"cause": x, "effect": y, "min_p": float(np.min(pvals))})
g_week = pd.DataFrame(rows).sort_values("min_p")
display(g_week)
g_week.to_csv(f"{TAB_W}/granger_minp_weekly.csv", index=False)


,cause,effect,min_p
2,BTC,DJI,0.000169
0,BTC,SPX,0.000773
1,ETH,SPX,0.002051
4,ETH,GOLD,0.018541
3,BTC,GOLD,0.033609


## WEEKLY — VAR → generalized FEVD connectedness (full sample)
Use H = 8 and maxlags = 8 (coarser horizon).


In [ ]:
fevdW, dirW, TCI_W = connectedness(rets_w[cols], H=8, maxlags=8)
fevdW.to_csv(f"{TAB_W}/fevd_full_weekly.csv", index=True)
dirW_reset = dirW.reset_index().rename(columns={"index": "Asset"})
dirW_reset.to_csv(f"{TAB_W}/directional_full_weekly.csv", index=False)
pd.DataFrame({"TCI":[TCI_W]}).to_csv(f"{TAB_W}/TCI_full_weekly.csv", index=False)

print("WEEKLY TCI:", round(TCI_W,2), "%")
display(dirW.sort_values("NET", ascending=False))

plt.figure(figsize=(6.2,5.0))
sns.heatmap(fevdW, annot=True, fmt=".2f", cmap="rocket", cbar_kws={"label":"GFEVD share"})
plt.title("Generalized FEVD (H=8) — Full Sample (Weekly)")
savefig(f"{FIG_W}/fevd_full_weekly.png")


WEEKLY TCI: 69.57 %


,TO_others,FROM_others,NET
GOLD,0.961505,0.076891,0.884614
ETH,0.570143,0.081005,0.489138
BTC,0.873664,0.440653,0.433011
DJI,0.616136,1.195666,-0.579530
SPX,0.456970,1.684203,-1.227233


## WEEKLY — Rolling connectedness
Weekly series are shorter; use window ≈ 104 weeks (~2y), step = 4.


In [ ]:
WINDOW_W, STEP_W, H_W, MAXLAGS_W = 104, 4, 8, 8
datesW, tci_valsW = [], []
netW = {c: [] for c in cols}

for i in tqdm(range(WINDOW_W, len(rets_w), STEP_W)):
    sub = rets_w.iloc[i-WINDOW_W:i]
    try:
        _, dir_t, tci_t = connectedness(sub, H=H_W, maxlags=MAXLAGS_W)
        datesW.append(rets_w.index[i])
        tci_valsW.append(tci_t)
        for c in cols:
            netW[c].append(dir_t.loc[c, "NET"])
    except Exception:
        continue

rollW = pd.DataFrame({"date":datesW, "TCI":tci_valsW}).set_index("date")
for c in cols:
    rollW[f"NET_{c}"] = netW[c]
rollW.to_csv(f"{TAB_W}/rolling_connectedness_weekly.csv")

plt.figure(figsize=(10,4))
rollW["TCI"].plot()
plt.title(f"Rolling TCI (Weekly, window={WINDOW_W}, step={STEP_W})")
plt.ylabel("%")
savefig(f"{FIG_W}/rolling_TCI_weekly.png")

plt.figure(figsize=(10,6))
rollW[[f"NET_{c}" for c in cols]].plot(ax=plt.gca(), lw=1)
plt.axhline(0, ls="--", lw=1)
plt.title("Rolling NET Spillovers (Weekly)")
savefig(f"{FIG_W}/rolling_NET_weekly.png")


100%|██████████| 27/27 [00:01<00:00, 20.80it/s]


## WEEKLY — Time-scale connectedness (SWT, db2, 3 levels)


In [ ]:
# WEEKLY — Time-scale connectedness (SWT) — fixed/robust
DESIRED_LEVELS = 3
weekly_base = rets_w[cols].dropna()

weekly_trim, LEVELS_EFF_W = prep_for_swt_df(weekly_base, DESIRED_LEVELS)
print(f"Weekly SWT levels used: {LEVELS_EFF_W}; "
      f"trimmed to length {len(weekly_trim)} (multiple of {2**LEVELS_EFF_W}).")

tci_lvlsW = []
for L in range(1, LEVELS_EFF_W + 1):
    comp = {}
    for c in cols:
        D = swt_details_series(weekly_trim[c], levels=LEVELS_EFF_W, wavelet="db2")
        comp[c] = D[L-1]
    dfL = pd.DataFrame(comp, index=weekly_trim.index).dropna()

    fevdL, dirL, tciL = connectedness(dfL, H=8, maxlags=8)
    tci_lvlsW.append((f"L{L}", float(tciL)))

    fevdL.to_csv(f"{TAB_W}/fevd_scale_L{L}_weekly.csv", index=True)
    dirL_reset = dirL.reset_index().rename(columns={"index": "Asset"})
    dirL_reset.to_csv(f"{TAB_W}/directional_scale_L{L}_weekly.csv", index=False)
    pd.DataFrame({"TCI": [tciL]}).to_csv(f"{TAB_W}/TCI_scale_L{L}_weekly.csv", index=False)

if tci_lvlsW:
    levelsW, tcisW = zip(*tci_lvlsW)
    plt.figure(figsize=(5.5,3.8))
    plt.bar(levelsW, tcisW)
    plt.title("Connectedness by Time Scale (Weekly)")
    plt.ylabel("%")
    savefig(f"{FIG_W}/TCI_by_scale_weekly.png")
else:
    print("No admissible SWT levels; skipping weekly scale plot.")


Weekly SWT levels used: 1; trimmed to length 208 (multiple of 2).


## Export Results
Creates `results_daily.zip` and `results_weekly.zip` with all figures + tables for LaTeX.


In [ ]:
!zip -r -q results_daily.zip  {FIG_D} {TAB_D}
!zip -r -q results_weekly.zip {FIG_W} {TAB_W}
print("Exported: results_daily.zip and results_weekly.zip")


Exported: results_daily.zip and results_weekly.zip


In [ ]:
# Summary cell (robust to 'Unnamed: 0' vs 'Asset' and variable wavelet levels)
import os, json, glob
import pandas as pd

ROOT  = "expanded_outputs"
TAB_D = f"{ROOT}/tables_results_daily"
TAB_W = f"{ROOT}/tables_results_weekly"

def safe_read(path):
    return pd.read_csv(path) if os.path.exists(path) else None

def normalize_dir_df(df):
    """Ensure directional table has 'Asset' column; if not, try to derive/rename."""
    if df is None:
        return None
    cols = list(df.columns)
    if "Asset" in cols:
        return df
    if "Unnamed: 0" in cols:
        df = df.rename(columns={"Unnamed: 0": "Asset"})
        return df
    # If neither exists but there's an index with names, move index to column
    if df.index.name is not None or any(df.index.astype(str) != pd.RangeIndex(len(df)).astype(str)):
        df = df.reset_index().rename(columns={"index": "Asset"})
    return df

def read_scale_tcIs(folder, suffix):
    """Discover all TCI_scale files (any levels) and return dict level->TCI."""
    out = {}
    pattern = os.path.join(folder, f"TCI_scale_L*_{suffix}.csv")
    for p in sorted(glob.glob(pattern)):
        try:
            level = os.path.basename(p).split("_")[2]  # 'L1' from 'TCI_scale_L1_*.csv'
            tci = float(pd.read_csv(p)["TCI"].iloc[0])
            out[level] = tci
        except Exception:
            continue
    return out

def top_bottom_net(df, k=5):
    """Return top and bottom by NET with Asset/TO/FROM/NET rounded."""
    if df is None or "NET" not in df.columns:
        return None
    df2 = df.copy()
    # Ensure column names exist and are float
    for col in ["TO_others", "FROM_others", "NET"]:
        if col in df2.columns:
            df2[col] = pd.to_numeric(df2[col], errors="coerce")
    s = df2.sort_values("NET", ascending=False)
    def _pack(rows):
        out = []
        for _, r in rows.iterrows():
            out.append({
                "Asset": r.get("Asset", None),
                "TO_others": None if pd.isna(r.get("TO_others", None)) else float(r["TO_others"]),
                "FROM_others": None if pd.isna(r.get("FROM_others", None)) else float(r["FROM_others"]),
                "NET": None if pd.isna(r.get("NET", None)) else float(r["NET"]),
            })
        return out
    return {"top": _pack(s.head(k)), "bottom": _pack(s.tail(k))}

summary = {}

# --- Daily ---
tci_daily_df = safe_read(f"{TAB_D}/TCI_full_daily.csv")
dir_daily_df = normalize_dir_df(safe_read(f"{TAB_D}/directional_full_daily.csv"))
g_daily_df   = safe_read(f"{TAB_D}/granger_minp_daily.csv")
scale_daily  = read_scale_tcIs(TAB_D, "daily")

summary["TCI_daily"] = None if tci_daily_df is None else float(tci_daily_df["TCI"].iloc[0])
summary["NET_daily"] = top_bottom_net(dir_daily_df, k=5)
summary["Granger_daily"] = (None if g_daily_df is None
                            else g_daily_df.sort_values("min_p").to_dict(orient="records"))
summary["Scale_TCI_daily"] = scale_daily

# --- Weekly ---
tci_weekly_df = safe_read(f"{TAB_W}/TCI_full_weekly.csv")
dir_weekly_df = normalize_dir_df(safe_read(f"{TAB_W}/directional_full_weekly.csv"))
g_weekly_df   = safe_read(f"{TAB_W}/granger_minp_weekly.csv")
scale_weekly  = read_scale_tcIs(TAB_W, "weekly")

summary["TCI_weekly"] = None if tci_weekly_df is None else float(tci_weekly_df["TCI"].iloc[0])
summary["NET_weekly"] = top_bottom_net(dir_weekly_df, k=5)
summary["Granger_weekly"] = (None if g_weekly_df is None
                             else g_weekly_df.sort_values("min_p").to_dict(orient="records"))
summary["Scale_TCI_weekly"] = scale_weekly

# Pretty print + save JSON
print(json.dumps(summary, indent=2))
with open(os.path.join(ROOT, "summary_results.json"), "w") as f:
    json.dump(summary, f, indent=2)
print("\nSaved to:", os.path.join(ROOT, "summary_results.json"))


{
  "TCI_daily": 51.47237616671268,
  "NET_daily": {
    "top": [
      {
        "Asset": "GOLD",
        "TO_others": 0.4363156979927083,
        "FROM_others": 0.054003407268258,
        "NET": 0.3823122907244503
      },
      {
        "Asset": "SPX",
        "TO_others": 0.5716707462953445,
        "FROM_others": 0.6223822793485216,
        "NET": -0.0507115330531771
      },
      {
        "Asset": "DJI",
        "TO_others": 0.5689880129194279,
        "FROM_others": 0.6198133370299158,
        "NET": -0.0508253241104879
      },
      {
        "Asset": "BTC",
        "TO_others": 0.4986951114958387,
        "FROM_others": 0.6289282658235372,
        "NET": -0.1302331543276985
      },
      {
        "Asset": "ETH",
        "TO_others": 0.4979492396323139,
        "FROM_others": 0.6484915188654007,
        "NET": -0.1505422792330867
      }
    ],
    "bottom": [
      {
        "Asset": "GOLD",
        "TO_others": 0.4363156979927083,
        "FROM_others": 0.05400340726825

In [ ]:
!zip -r -q /content/results_all.zip expanded_outputs
from google.colab import files
files.download('/content/results_all.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>